In [1]:
import os
import zipfile
import sys
import mysql.connector
import requests
import re
import shutil


sys.path.append('/home/lib')
nodeServer = os.environ['NODE_SERVER']
db_host = os.environ['SQL_HOST']
db_user = os.environ['SQL_USER']
db_password = os.environ['SQL_PASSWORD']
database = os.environ['DATABASE']

# MySQL 서버에 연결
try:
    conn = mysql.connector.connect(
        host=db_host,
        user=db_user,
        password=db_password,
        database=database
    )
    print("MySQL에 정상적으로 연결되었습니다.")
    cur = conn.cursor()

except mysql.connector.Error as e:
    print(f"MySQL 연결 오류: {e}")

try:
    stylingSuggestionProductList = """
        SELECT ss.styling_suggestion_id
         , ssp.styling_suggestion_product_id
         , CONCAT('https://dev-image.con-f.co.kr/', imf.image_path)
         , ssp.hyperlink
FROM styling_suggestion AS ss
             INNER JOIN styling_suggestion_product AS ssp
             ON ss.styling_suggestion_id = ssp.styling_suggestion_id
             INNER JOIN image_file AS imf
             ON imf.ref_unique_id = ssp.styling_suggestion_product_id AND
                imf.image_file_group_id = 7
            WHERE ss.confirm_yn = 'N'
              AND ss.send_date >= DATE_SUB(NOW(), INTERVAL 30 DAY)
              AND ss.styling_suggestion_status_code = '20')
    """
            
    cur.execute(stylingSuggestionProductList)

    # fetchall을 사용하여 모든 결과 행을 가져옴
    imageList = cur.fetchall()

    os.system(f'rm -r ./제품이미지다운로드')
    download_folder_path = "./제품이미지다운로드"

    for row in imageList:
        styling_suggestion_id, styling_suggestion_product_id, image_path, hyperlink= row

        # styling_suggestion_id로 폴더 생성
        folder_path = os.path.join(download_folder_path, str(styling_suggestion_id))
        os.makedirs(folder_path, exist_ok=True)

        # image_path로부터 이미지 다운로드 및 저장
        response = requests.get(image_path, stream=True)
        with open(os.path.join(folder_path, f"{styling_suggestion_product_id}.jpg"), 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    file.write(chunk)

        # info.txt 파일 생성 및 내용 추가
        with open(os.path.join(folder_path, "info.txt"), 'a', encoding='utf-8') as info_file:
            info_file.write(f"styling_suggestion_product_id: {styling_suggestion_product_id}, hyperlink: {hyperlink}\n")

    # 전체 다운로드 폴더를 압축
    shutil.make_archive(download_folder_path, 'zip', download_folder_path)

except Exception as e:
    print(f"에러 발생: {e}")

finally:
    conn.close()

MySQL에 정상적으로 연결되었습니다.
